In [1]:
#| default_exp 40_ngame-for-amazon131-with-input-concatenation

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import os,torch, torch.multiprocessing as mp, pickle, numpy as np
from transformers import AutoTokenizer
from tqdm.auto import tqdm

from xcai.basics import *
from xcai.models.PPP0XX import DBT009,DBT011

In [4]:
os.environ['WANDB_MODE'] = 'disabled'

In [5]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='medic_09-amazon131'

## Helper

In [6]:
#| export
def separate_title_and_content(input_text):
    title,content = [],[]
    for o in tqdm(input_text):
        text = o.split('  ', maxsplit=1)

        if len(text) == 2: 
            t,c = text
        else: 
            t,c = text[0], ''

        title.append(t)
        content.append(c)
    return title, content
    

In [7]:
#| export
def tokenize(tokenizer, info):
    o = tokenizer(info['input_text'], truncation=True, max_length=32)
    info.update(o)
    
    o = tokenizer(info['content_text'], truncation=True, max_length=1024)
    
    info['content_input_ids'] = o['input_ids']
    info['content_attention_mask'] = o['attention_mask']


In [8]:
#| export
def append_sep_token(input_ids, attention_mask):
    sep_tok = input_ids[0][-1]
    
    new_input_ids = [o+[sep_tok] for o in input_ids]
    new_attention_mask = [o+[1] for o in attention_mask]
    
    return new_input_ids, new_attention_mask


def append_sep_token_into_block(block):
    dset_type = ['train', 'test']
    info_type = ['data_info', 'lbl_info']
    
    for dt in dset_type:
        dset = getattr(block, dt)
        for it in info_type:
            info = getattr(dset.dset.data, it)
            input_ids, attention_mask = append_sep_token(info['input_ids'], info['attention_mask'])
            info['input_ids'],info['attention_mask'] = input_ids, attention_mask
            

In [9]:
#| export
def concatenate_title_and_content_ids(info, max_input_length, exclude_sep_tok=True):
    n_data = len(info['input_ids'])
    sep_tok = info['input_ids'][0][-1]
    
    for i,(p,q) in tqdm(enumerate(zip(info['input_ids'],info['content_input_ids'])), total=n_data):
        input_ids = p[:-1] + q[1:] if exclude_sep_tok else p + q[1:]
        if len(input_ids) > max_input_length:
            input_ids = input_ids[:max_input_length-1] + [sep_tok]
        info['input_ids'][i] = input_ids
    
    for i,(p,q) in tqdm(enumerate(zip(info['attention_mask'],info['content_attention_mask'])), total=n_data):
        attention_mask = p[:-1] + q[1:] if exclude_sep_tok else p + q[1:]
        if len(attention_mask) > max_input_length:
            attention_mask = attention_mask[:max_input_length-1] + [1]
        info['attention_mask'][i] = attention_mask
        

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    build_block = False
    data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data'
    pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'

    """ Load data """    
    pkl_file = f'{pkl_dir}/processed/amazon131_data-meta_distilbert-base-uncased_xcs_cat-512.pkl'
    if build_block:
        block = XCBlock.from_cfg(data_dir, 'data_meta', dset='amazontitles131', transform_type='xcs', tokenizer='distilbert-base-uncased', 
                                 sampling_features=[('lbl2data',1)], oversample=False)

        train_title, train_content = separate_title_and_body(block.train.dset.data.data_info['input_text'])
        test_title, test_content = separate_title_and_body(block.test.dset.data.data_info['input_text'])

        block.train.dset.data.data_info['input_text'] = train_title
        block.train.dset.data.data_info['content_text'] = train_content
        
        block.test.dset.data.data_info['input_text'] = test_title
        block.test.dset.data.data_info['content_text'] = test_content

        tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        tokenize(tokenizer, block.train.dset.data.data_info)
        tokenize(tokenizer, block.test.dset.data.data_info)

        append_sep_token_into_block(block)

        block = AugmentMetaInputIdsTfm.apply(block, 'cat_meta', 'data', 256, True)
        block = AugmentMetaInputIdsTfm.apply(block, 'cat_meta', 'lbl', 256, True)

        block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_cat']
        block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_cat']
        block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_cat']
        block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_cat']
        
        block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_aug_cat']
        block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_aug_cat']
        block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_aug_cat']
        block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_aug_cat']

        concatenate_title_and_content_ids(block.train.dset.data.data_info, 512, exclude_sep_tok=False)
        concatenate_title_and_content_ids(block.test.dset.data.data_info, 512, exclude_sep_tok=False)
        
        with open(pkl_file, 'wb') as file: pickle.dump(block, file)
    else:
        with open(pkl_file, 'rb') as file: block = pickle.load(file)

    block.train.dset.meta = {}
    block.test.dset.meta = {}
    
    """ Training Arguements """
    args = XCLearningArguments(
        output_dir='/home/scai/phd/aiz218323/scratch/outputs/medic/40_ngame-for-amazon131-with-input-concatenation',
        logging_first_step=True,
        per_device_train_batch_size=800,
        per_device_eval_batch_size=800,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        representation_search_type='BRUTEFORCE',
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
        
        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,
        
        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
        
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
    )

    metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                      pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

    """ Model """
    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
    model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                                   n_negatives=10, apply_softmax=True, use_encoder_parallel=True)
    model.init_dr_head()

    
    learn = XCLearner(
        model=model, 
        args=args,
        train_dataset=block.train.dset,
        eval_dataset=block.test.dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    mp.freeze_support()
    learn.train()
    

## Prediction

In [ ]:
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    fp16=True,
)

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
output_dir = f"/home/scai/phd/aiz218323/scratch/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1,
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from safetensors import safe_open

model_weight_file = f'{mname}/model.safetensors'

model_weights = {}
with safe_open(model_weight_file, framework="pt") as file:
    for k in file.keys(): model_weights[k] = file.get_tensor(k)
        

In [ ]:
model.load_state_dict(model_weights, strict=False)

_IncompatibleKeys(missing_keys=['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
o = learn.predict(block.test.dset)

  0%|          | 0/196 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,24.1771,16.1361,12.2884,7.9308,24.1771,24.4573,25.4609,27.2161,20.1098,21.5063,23.1904,26.8649,20.1098,22.0686,23.477,25.37,31.8421,46.4144,49.9468,0.0294,262.9244,675.156,0.422


In [ ]:
mname

'/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2/checkpoint-85000'

In [ ]:
pred_dir = f"{mname}/predictions/"
os.makedirs(pred_dir, exist_ok=True)

with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(o, file)